## MATRIX INVERSION AS A QUBO PROBLEM

Ref  
[1.Floating-Point Calculations on a Quantum Annealer:
Division and Matrix Inversion](https://arxiv.org/pdf/1901.06526.pdf)

In this section we present an algorithm for solving a system of linear equations on a
quantum annealer. To precisely define the mathematical problem, let $M$ be a nonsingular $N × N$ real matrix, and let $Y$ be a real $N$ dimensional vector; we then wish to solve the linear equation
$$ M · x = Y$$
The linearity of the system means that there is a unique solution,
$$x = M^{−1} · Y$$

Constructing a quadratic matrix 
$$H(x) = (M x − Y)^2 = (M x − Y)^T · (M x − Y)$$
$$H(x) = x^T M^T Mx - x^T M^T Y - Y^T M x +Y^T Y$$
$$H(x) = \sum_{ijk=1}^{N} M_{ki} M_{kj} x^i x^j - 2\sum_{ij=1}^N Y_j M_{ji} x^i + ||Y^2||$$
To obtain a floating point representation of each component
of $x = (x_1, · · · , x_N)^T$ by expanding in powers of 2 multiplied by Boolean-valued variables
$$\chi^i = \sum_{r=0}^{R-1} 2^{-r}q_r^i $$
$$x^i = 2\chi^i-1$$

And to obtain integer representation, the real value $x_i$

$$ x_i = \sum_{l=-m}^m 2^l q_{i,l}^+ - \sum_{l=-m}^m 2^l q_{i,l}^- $$

To present both position and negetive numbers $q_{i,l}^+$ and $q_{i,l}^-$ are involved.

As before, the domains are given by $\chi^i\in [0, 2)$ and $x^i \in [−1, 3)$, and upon expressing $x$ as a function the $q_r^i$ we can recast $H(x)$ in the form  
$$H(q) = \sum_{i=1}^{N}\sum_{r=0}^{R-1} a_r^i q_r^j + \sum_{i=1}^N \sum_{i\neq j=1}^N \sum_{r=0}^{R-1} \sum_{s=0}^{R-1} b_{rs}^{ij} q_{r}^i q_s^i$$

In [63]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import random, math
import copy

Dimension = 3
qubits = 4
# A = np.array([[3, 1], [-1, 2]])
# b = np.array([-1, 5])
A = np.array([[1, 2, -3], [2, -5, 4],[5, 4, -1]])
b = np.array([-3, 13, 5])

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
# print("QM" + str(QM));
for k in range(Dimension): 
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*b[k]
            po1 = (qubits*2)*i + l
            po2 = (qubits*2)*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2

# print("after calculating linear terms: ")
# print(QM)

for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = (qubits*2)*i + l1
                po2 = (qubits*2)*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = (qubits*2)*i + l1 + qubits
                po4 = (qubits*2)*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef

# print("after calculating quadratic terms: ")
# print(QM)

for k in range(Dimension):
    for i in range(Dimension-1):
            for j in range(i+1,Dimension):
                for l1 in range(qubits):
                    for l2 in range(qubits):
                        qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j]
                        po1 = (qubits*2)*i + l1
                        po2 = (qubits*2)*j + l2
                        QM[po1][po2] = QM[po1][po2] + qcef
                        po3 = (qubits*2)*i + l1 + qubits
                        po4 = (qubits*2)*j + l2 + qubits
                        QM[po3][po4] = QM[po3][po4] + qcef
                        po5 = (qubits*2)*i + l1
                        po6 = (qubits*2)*j + l2 + qubits
                        QM[po5][po6] = QM[po5][po6] - qcef
                        po7 = (qubits*2)*i + l1 + qubits
                        po8 = (qubits*2)*j + l2
                        QM[po7][po8] = QM[po7][po8] - qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)

# Matrix Q is
[[  -66.   120.   240.   480.     0.     0.     0.     0.    24.    48.
     96.   192.   -24.   -48.   -96.  -192.     0.     0.     0.     0.
      0.     0.     0.     0.]
 [    0.   -72.   480.   960.     0.     0.     0.     0.    48.    96.
    192.   384.   -48.   -96.  -192.  -384.     0.     0.     0.     0.
      0.     0.     0.     0.]
 [    0.     0.    96.  1920.     0.     0.     0.     0.    96.   192.
    384.   768.   -96.  -192.  -384.  -768.     0.     0.     0.     0.
      0.     0.     0.     0.]
 [    0.     0.     0.  1152.     0.     0.     0.     0.   192.   384.
    768.  1536.  -192.  -384.  -768. -1536.     0.     0.     0.     0.
      0.     0.     0.     0.]
 [    0.     0.     0.     0.   126.   120.   240.   480.   -24.   -48.
    -96.  -192.    24.    48.    96.   192.     0.     0.     0.     0.
      0.     0.     0.     0.]
 [    0.     0.     0.     0.     0.   312.   480.   960.   -48.   -96.
   -192.  -384.    48.    96.   192.   

## Apply QUBO

In [64]:
qbit_list = []
for val in range(len(QM)):
    qbit_list.append('q'+str(val+1)) 
print(qbit_list)
Q2 = {}
for i in range(len(QM)):
    for j in range(len(QM)):
        if QM[i][j] != 0:
            Q2[(qbit_list[i],qbit_list[j])] = QM[i][j]
            
print(Q2)
            

['q1', 'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10', 'q11', 'q12', 'q13', 'q14', 'q15', 'q16', 'q17', 'q18', 'q19', 'q20', 'q21', 'q22', 'q23', 'q24']
{('q1', 'q1'): -66.0, ('q1', 'q2'): 120.0, ('q1', 'q3'): 240.0, ('q1', 'q4'): 480.0, ('q1', 'q9'): 24.0, ('q1', 'q10'): 48.0, ('q1', 'q11'): 96.0, ('q1', 'q12'): 192.0, ('q1', 'q13'): -24.0, ('q1', 'q14'): -48.0, ('q1', 'q15'): -96.0, ('q1', 'q16'): -192.0, ('q2', 'q2'): -72.0, ('q2', 'q3'): 480.0, ('q2', 'q4'): 960.0, ('q2', 'q9'): 48.0, ('q2', 'q10'): 96.0, ('q2', 'q11'): 192.0, ('q2', 'q12'): 384.0, ('q2', 'q13'): -48.0, ('q2', 'q14'): -96.0, ('q2', 'q15'): -192.0, ('q2', 'q16'): -384.0, ('q3', 'q3'): 96.0, ('q3', 'q4'): 1920.0, ('q3', 'q9'): 96.0, ('q3', 'q10'): 192.0, ('q3', 'q11'): 384.0, ('q3', 'q12'): 768.0, ('q3', 'q13'): -96.0, ('q3', 'q14'): -192.0, ('q3', 'q15'): -384.0, ('q3', 'q16'): -768.0, ('q4', 'q4'): 1152.0, ('q4', 'q9'): 192.0, ('q4', 'q10'): 384.0, ('q4', 'q11'): 768.0, ('q4', 'q12'): 1536.0, ('q4', 'q13'): 

In [65]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

# linear = {('q1','q1'):26.0, ('q2','q2'):72.0, ('q3','q3'):-6.0, ('q4','q4'):8.0, ('q5','q5'):-13.0, ('q6','q6'):-16.0, ('q7','q7'):23.0, ('q8','q8'):56.0}

# quadratic = {('q1','q2'):40.0, ('q1','q5'):2.0, ('q1','q6'):4.0, ('q1','q7'):-2.0, ('q1','q8'):-4.0, ('q2','q5'):4.0, ('q2','q6'):8.0, ('q2','q7'):-4.0, ('q2','q8'):-8.0, ('q3','q4'):40.0, ('q3','q5'):-2.0, ('q3','q6'):-4.0, ('q3','q7'):2.0, ('q3','q8'):4.0, ('q4','q5'):-4.0, ('q4','q6'):-8.0, ('q4','q7'):4.0, ('q4','q8'):8.0, ('q5','q6'):20.0, ('q7','q8'):20.0}

# Q = dict(linear)
# Q.update(quadratic)

# print(Q)

sampleset = sampler_auto.sample_qubo(Q2, num_reads=1000)


### Convert Qbit to Decimal 

In [66]:
samples = sampleset.samples()
# type(samples[0])
solution_sample = samples[0]

res = []

for i in range(Dimension):
    res.append(0)
    for j in range(qubits):
        pos1 = i*2*qubits+ j
        pos2 = i*2*qubits+ j + qubits
        if qbit_list[pos1] in solution_sample:
            res[i] += (pow(2,j)*solution_sample[qbit_list[pos1]])
        if qbit_list[pos2] in solution_sample:
            res[i] -= (pow(2,j)*solution_sample[qbit_list[pos2]])


# for i in range(Dimension):
#     res.append(0)
#     for j in range(qubits):
#         res[i] += (pow(2,j)*qbit_per_var[i][j] -  pow(2,j)*qbit_per_var[i][qubits+j])

print("Output: ", res)
    


Output:  [2, -1, 1]


In [67]:
import dimod

#print(Q)
J = dimod.qubo_to_ising(Q2)
print(J)

({'q1': 177.0, 'q2': 354.0, 'q3': 708.0, 'q4': 1416.0, 'q9': 388.5, 'q10': 777.0, 'q11': 1554.0, 'q12': 3108.0, 'q13': 286.5, 'q14': 573.0, 'q15': 1146.0, 'q16': 2292.0, 'q5': 273.0, 'q6': 546.0, 'q7': 1092.0, 'q8': 2184.0, 'q17': 139.0, 'q18': 278.0, 'q19': 556.0, 'q20': 1112.0, 'q21': 251.0, 'q22': 502.0, 'q23': 1004.0, 'q24': 2008.0}, {('q1', 'q2'): 30.0, ('q1', 'q3'): 60.0, ('q1', 'q4'): 120.0, ('q1', 'q9'): 6.0, ('q1', 'q10'): 12.0, ('q1', 'q11'): 24.0, ('q1', 'q12'): 48.0, ('q1', 'q13'): -6.0, ('q1', 'q14'): -12.0, ('q1', 'q15'): -24.0, ('q1', 'q16'): -48.0, ('q2', 'q3'): 120.0, ('q2', 'q4'): 240.0, ('q2', 'q9'): 12.0, ('q2', 'q10'): 24.0, ('q2', 'q11'): 48.0, ('q2', 'q12'): 96.0, ('q2', 'q13'): -12.0, ('q2', 'q14'): -24.0, ('q2', 'q15'): -48.0, ('q2', 'q16'): -96.0, ('q3', 'q4'): 480.0, ('q3', 'q9'): 24.0, ('q3', 'q10'): 48.0, ('q3', 'q11'): 96.0, ('q3', 'q12'): 192.0, ('q3', 'q13'): -24.0, ('q3', 'q14'): -48.0, ('q3', 'q15'): -96.0, ('q3', 'q16'): -192.0, ('q4', 'q9'): 48.0, ('

In [68]:
sampleset = sampler_auto.sample_ising(J[0], J[1], num_reads=1000)
print(sampleset)

 -1  -1  +1  -1  +1  -1  -1  -1 +1 ... -1 -13137.0       3 ...
512 +1  -1  -1  -1  -1  +1  -1  +1  -1  -1  -1 +1 ... -1 -13137.0       1 ...
229 +1  -1  -1  -1  +1  +1  -1  +1  -1  -1  -1 +1 ... -1 -13134.0       1 ...
273 +1  -1  -1  -1  +1  +1  -1  +1  -1  -1  -1 -1 ... -1 -13134.0       1 ...
4   -1  -1  -1  -1  -1  +1  -1  +1  -1  -1  -1 +1 ... -1 -13111.0      10 ...
44  -1  -1  -1  -1  -1  +1  -1  +1  -1  -1  -1 +1 ... -1 -13111.0       2 ...
558 -1  -1  -1  -1  -1  +1  -1  +1  -1  -1  -1 +1 ... -1 -13111.0       1 ...
621 -1  -1  -1  -1  +1  -1  -1  +1  -1  -1  -1 +1 ... +1 -13111.0       1 ...
5   +1  -1  -1  -1  -1  +1  -1  +1  -1  -1  -1 -1 ... -1 -13105.0       3 ...
397 -1  -1  -1  -1  -1  -1  -1  +1  -1  -1  -1 +1 ... -1 -13105.0       1 ...
264 -1  -1  -1  -1  +1  -1  -1  +1  -1  -1  -1 +1 ... -1 -13090.0       2 ...
266 +1  -1  -1  -1  +1  +1  -1  +1  -1  -1  -1 -1 ... -1 -13078.0       1 ...
526 -1  -1  -1  -1  +1  -1  -1  +1  -1  -1  -1 +1 ... +1 -13075.0       1 ...
1